In [ ]:
using Distributions

In [ ]:
include("run_remotely.jl")

In [ ]:
function simulate(params)
    pdict = Dict(
        "p-drop-contact" => params[1], 
#        "speed-expl-stay" => params[2], 
#        "speed-expl-move" => params[3], 
        "p-info-contacts" => params[2], 
        "p-transfer-info" => params[3],
#        "error" => params[6], 
#        "error-risk" => params[7], 
#        "error-frict" => params[8], 
        "risk-scale" => params[4], 
#        "p-notice-death-c" => params[10], 
#        "p-notice-death-o" => params[11], 
#        "speed-risk-indir" => params[12], 
#        "speed-risk-obs" => params[13], 
#        "speed-expl-risk" => params[14], 
        "path-penalty-risk" => params[5])

    deps = join(params[6:8], " ")
    
    med_args = "'" * 
        "--wait 10 --warmup 100 " * 
        "--int-w 0.01 0.077 0.153 0.428 0.389 " * 
        "--int-e 0.01 0.1755 0.3719 0.4024 0.4626 " * 
        "--risks-w 0.01 0.023 0.020 0.029 0.048 " *
        "--risks-e 0.01 0.0021 0.0012 0.0028 0.0005 " * 
        "'"
    
    args = " -m ../map_med1.json -s ../departures '--warmup 100 --dep $deps' -s ../mediterranean $med_args -t 500"
    for (k, v) in pdict
        args *= " --$k $v" 
    end
    
    args *= " --rand-seed-sim $(rand(1:100000)) --rand-seed-world $(rand(1:100000))"
    
    xdirs = [readdir("incoming"); readdir("running"); readdir("done")]
    filter!(x -> startswith(x, "x"), xdirs)
    mx = 1
    for dir in xdirs
        m = match(r"x([0-9]+)_", dir)
        if m == nothing 
            continue
        end
        n = parse(Int, m.captures[1])
        if mx <= n
            mx = n+1
        end
    end
    
    prefix = "x$(mx)_$(Threads.threadid())_"
    server = "sotonhpc"
    dir = "Science/southampon/runs/mediterranean"
    
    results = run_batch(server, dir, prefix, [args])[1]

    results
end

function costs(results)
    arrivals_w = [0.657941144053786, 0.207848228494849, 0.599476123312513]
    arrivals_e = [0.184888393242086, 1.1670864036315, 1.87191304579482]
    mort_w = [0.022736973763885, 0.019768432868398, 0.028785488958991, 0.047819332348149]
    mort_e = [0.002051786330565, 0.00120836500419, 0.002757134481611, 0.000542481662592]
    interc_w = [1.43020016842651, 0.884092762025546, 0.525283057533685]
    interc_e = [0.513977915432265, 1.33085307063509, 2.38377037562013]

    d1 = sqrt(sum((arrivals_w .- results[1]) .^ 2))
    d2 = sqrt(sum((arrivals_e .- results[2]) .^ 2))
    d3 = sqrt(sum((mort_w .- results[3]) .^ 2))
    d4 = sqrt(sum((mort_e .- results[4]) .^ 2))
    d5 = sqrt(sum((interc_w .- results[5]) .^ 2))
    d6 = sqrt(sum((interc_e .- results[6]) .^ 2))
    
    d1 + d2 + d3 * 10 + d4 * 10 + d5 + d6
end

dist(params) = costs(simulate(params))

In [ ]:
using Distributions
using Statistics

mutable struct Particle
    params :: Vector{Float64}
    dist :: Float64
end


function limit(params, priors)
    [max(minimum(d), min(maximum(d), p)) for (p, d) in zip(params, priors)]
end
        

function abc(priors, dist_func, pop_size, p_rem, sigma, n_iters; verbose = false, parallel = false, scale_noise = false)
    particles = Particle[]
    new_particles = [Particle(map(rand, priors), Inf) for i in 1:pop_size]
        
    iter = 0
    
    while true
        iter += 1
        verbose && println("$iter: simulating $(length(new_particles)) new particles...")
        
        if parallel
            Threads.@threads for p in new_particles
                p.dist = dist_func(p.params)
            end        
        else
            for p in new_particles
                p.dist = dist_func(p.params)
            end        
        end
            
        # add new particles to old ones
        particles = [particles ; new_particles]
        
        if iter >= n_iters
            return particles
        end

        rem = floor(Int, p_rem * length(particles))
    
        verbose && println("removing $rem of $(length(particles))")
        # remove worst particles
        sort!(particles, by=p->p.dist)
        particles = particles[1:(end-rem)]
        
        verbose && println("distance: ", particles[1].dist, " ", particles[end].dist)
        
        
        if scale_noise
            std_dev = [std([p.params[i] for p in particles]) for i in eachindex(priors)]
            verbose && println("stdd: ", join(std_dev, ", "))
            n_sigma = sigma .* std_dev
        else
            n_sigma = sigma
        end
        
        noise = Normal.(0, n_sigma)
        
        s = particles[end].dist + particles[1].dist
        weights = cumsum([s - p.dist for p in particles])
        sel = Uniform(0, weights[end])

        empty!(new_particles)
        
        for i in 1:pop_size
            anc = particles[searchsortedfirst(weights, rand(sel))]
            params = anc.params .+ rand.(noise)
            params = limit(params, priors)
            push!(new_particles, Particle(params, Inf))
        end
    end
            
            
end

In [ ]:
priors = [Uniform(0, 1), Uniform(0, 1), Uniform(0, 1), Uniform(0, 20), Uniform(0, 1), 
    Uniform(0.2, 1.5), Uniform(0.2, 1.5), Uniform(0.2, 1.5)]
noise = [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]

In [ ]:
#p = rand.(priors)
#simulate(p)

In [ ]:
abc(priors, dist, 200, 0.5, noise, 10, verbose=true, scale_noise=true, parallel=true)

In [ ]:
abc_result = ans

In [ ]:
sorted = sort(abc_result, by=p->p.dist);

In [ ]:
real_p = abc_result[1:end-200]
real_p2 = abc_result[end-199:end]

In [ ]:
using Plots

In [ ]:
histogram(map(x->x.params[1], real_p), bins=20)

In [ ]:
histogram(map(x->x.params[2], real_p), bins=20)

In [ ]:
histogram(map(x->x.params[3], real_p), bins=20)

In [ ]:
plotly()
scatter(map(x->1.0-x.params[1], real_p), map(x->x.params[2], real_p), map(x->x.params[3], real_p), 
    xlabel="p_keep", ylabel="p_info", zlabel="p_transfer")

In [ ]:
histogram(map(x->x.params[4], real_p), bins=20)

In [ ]:
histogram(map(x->x.dist, abc_result[200:end]))